In [69]:
import configparser
import tweepy
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
import time
import re
import hashlib

In [31]:
# get tokens
config = configparser.ConfigParser()
config.read('configfile.ini')
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
# authenticate
auth = tweepy.OAuth2AppHandler(api_key, api_key_secret)

In [32]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
def get_tweets_dataframe(working_df, tweets, hashtag):
    #working_df=working_df.drop_duplicates()
    print(f'working on {hashtag}')
    index=len(working_df)
    for tweet in tweets: 
        working_df.loc[index,'tweet_id']=tweet.id
        working_df.loc[index,'created_at']=tweet.created_at
        working_df.loc[index,'user']=tweet.user.screen_name
        working_df.loc[index,'full_text']=tweet.full_text
        working_df.loc[index,'favorite_count']=tweet.favorite_count
        working_df.loc[index,'retweet_count']=tweet.retweet_count
        working_df.loc[index,'hashtags']=hashtag
        #print(working_df.loc[index,'created_at'])
        #working_df=working_df.drop_duplicates()
        index+=1
        working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)
        #time.sleep(1)
    working_df=working_df.drop_duplicates()
    working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
          
    return working_df

In [45]:
def get_tweets_from_general_tags(working_df, hashtag):
    #load and merge the general_tags
    print(f'working on {hashtag}')
    general_tags = ['#SaveLockwoodandCo', 'Lockwood and Co']
    general_tags_df=pd.DataFrame()
    for tag in general_tags:
        try:
            general_tags_df=pd.concat([working_df, pd.read_csv(f'tweeterhashtags/{tag}.csv')], ignore_index=True)
        except:
            pass
    general_tags_df =general_tags_df.drop_duplicates()
    #general_tags_df['full_text'] = general_tags_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    #Get the rows in the working_df that have the hashtag inside the full_text column
    hashtag_df = general_tags_df[general_tags_df['full_text'].apply(lambda x: hashtag.lower() in x.lower())]
    #hashtag_df = hashtag_df.reset_index(drop=True)
    hashtag_df['hashtags'] = hashtag
    working_df=pd.concat([working_df, hashtag_df], ignore_index=True)
    working_df=working_df.drop_duplicates()
    working_df['full_text'] = working_df['full_text'].apply(lambda x: x.replace('\n',' ').replace(':', ' '))
    working_df.to_csv(f'tweeterhashtags/{hashtag}.csv', index=False)
    working_df['created_at'] = pd.to_datetime(working_df['created_at'], utc=True)
    # Group the DataFrame by day
    grouped_df = working_df.groupby(pd.Grouper(key='created_at', freq='D'))

    # Calculate the count of rows for each day
    count_per_day = grouped_df['hashtags'].count()

    # Display the count per day
    print(count_per_day)
  
    return working_df

In [71]:
# Create a hash function to generate anonymized values
def anonymize(value):
    # Convert the value to a string and hash it using SHA256 algorithm
    hashed_value = hashlib.sha256(str(value).encode()).hexdigest()
    # Take the first 8 characters of the hash as the anonymized value
    anonymized_value = hashed_value[:8]
    return anonymized_value

In [43]:
hashtagsweek1 = ['#FridayNightatPortlandRow', '#HauntedWatchParty', '#WatchPartyatPortlandRow', '#HauntedbyaType3','#TogetherForLockwoodandCo','#PrimeForLockwoodandCo','#BringBackLockwoodandCo']
hashtagsweek2 = ['#GhostHuntersWatchParty', '#DisneyForLockwoodandCo', '#BBCforLockwoodandCo', '#AppleTVforLockwoodandCo', '#PrimeForLockwoodandCo', '#JustRecklessEnough']
hashtagsweek3 = ['#LockwoodGhostAuditions', '#ParamountForLockwoodandCo', '#ScullandCo','#RapiersReady', '#CaringforCarlyle', '#DEPRACisOnTheWay', '#BunsForBunchurch']
hashtagsweek4 = ['#CompleteFictionAppreciation', '#DisneySaveLockwood', '#ArtistryofLockwoodandCo', '#ParamountSaveLockwood', '#GhostStrike', '#LockwoodParallelFandoms', '#JustRecklessEnough']
hashtagsweek5 = ['#StroudsAppreciation' '#VoteLockwoodforNFA', '#PrimeSaveLockwood', '#ScreamingStaircase', '#DEPRACrollcall', '#LivingforLockwood', '#RapiersReady']
hashtagsall = hashtagsweek1+hashtagsweek2+hashtagsweek3+hashtagsweek4 + hashtagsweek5
hashtagsall = list(set(hashtagsall))

In [35]:
'''
phrase = 'Lockwood and Co'
try:
    working_df = pd.read_csv(f'tweeterhashtags/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=phrase, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)

hashtag = '#SaveLockwoodandCo'
try:
    working_df = pd.read_csv(f'tweeterhashtags/{phrase}.csv')
    print(len(working_df))
except:
    working_df = pd.DataFrame(columns=['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count', 'retweet_count', 'hashtags'])
    
tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
working_df = get_tweets_dataframe(working_df, tweets, phrase)
    
working_df.drop_duplicates(inplace=True)
print(len(working_df))
    
time.sleep(60)
'''

19597


Rate limit reached. Sleeping for: 401
Rate limit reached. Sleeping for: 373
Rate limit reached. Sleeping for: 311


19990
created_at
2023-06-02 00:00:00+00:00     128
2023-06-03 00:00:00+00:00     626
2023-06-04 00:00:00+00:00     545
2023-06-05 00:00:00+00:00     885
2023-06-06 00:00:00+00:00    1447
2023-06-07 00:00:00+00:00    1832
2023-06-08 00:00:00+00:00    2316
2023-06-09 00:00:00+00:00    3419
2023-06-10 00:00:00+00:00    3711
2023-06-11 00:00:00+00:00    4475
2023-06-12 00:00:00+00:00     606
Freq: D, Name: hashtags, dtype: int64


In [40]:
for hashtag in hashtagsweek5:
    try:
        working_df=pd.read_csv(f'tweeterhashtags/{hashtag}.csv')
        #working_df = pd.read_csv('tweeterhashtags/#SkullandCo.csv')
        print(len(working_df))
    except:
        working_df=pd.DataFrame(columns=['tweet_id','created_at', 'user', 'full_text','favorite_count','retweet_count','hashtags'])
    tweets = tweepy.Cursor(api.search_tweets, q=hashtag, tweet_mode='extended').items()
    working_df = get_tweets_dataframe(working_df, tweets, hashtag)
    working_df=working_df.drop_duplicates()
    print(len(working_df))
    time.sleep(60)

#StroudsAppreciation#VoteLockwoodforNFA
0
Series([], Freq: D, Name: hashtags, dtype: int64)
#PrimeSaveLockwood
0
Series([], Freq: D, Name: hashtags, dtype: int64)
#ScreamingStaircase
0
Series([], Freq: D, Name: hashtags, dtype: int64)
#DEPRACrollcall
0
Series([], Freq: D, Name: hashtags, dtype: int64)
#LivingforLockwood
0
Series([], Freq: D, Name: hashtags, dtype: int64)
#RapiersReady
5248
5251
created_at
2023-05-28 00:00:00+00:00       1
2023-05-29 00:00:00+00:00       0
2023-05-30 00:00:00+00:00       0
2023-05-31 00:00:00+00:00      33
2023-06-01 00:00:00+00:00    4110
2023-06-02 00:00:00+00:00    1018
2023-06-03 00:00:00+00:00      58
2023-06-04 00:00:00+00:00       4
2023-06-05 00:00:00+00:00       2
2023-06-06 00:00:00+00:00       0
2023-06-07 00:00:00+00:00       4
2023-06-08 00:00:00+00:00      10
2023-06-09 00:00:00+00:00       0
2023-06-10 00:00:00+00:00       0
2023-06-11 00:00:00+00:00       3
2023-06-12 00:00:00+00:00       8
Freq: D, Name: hashtags, dtype: int64


In [ ]:
for hashtag in hashtagsall:
    try:
        working_df=pd.read_csv(f'tweeterhashtags/{hashtag}.csv')
        working_df = get_tweets_from_general_tags(working_df, hashtag)
    except:
        pass

In [47]:
working_df=pd.DataFrame()
for hashtag in hashtagsall:
    try:
        working_df=pd.concat([working_df, pd.read_csv(f'tweeterhashtags/{hashtag}.csv')], ignore_index=True)
    except:
        pass
working_df

,tweet_id,created_at,user,full_text,favorite_count,retweet_count,hashtags,tweet_df
0,1.663313e+18,2023-05-29 22:33:42+00:00,CayleeGuy,RT @aDEPRACofficial Good morning Locknation! ...,0.0,38.0,#BBCforLockwoodandCo,NaN
1,1.663313e+18,2023-05-29 22:33:30+00:00,CayleeGuy,RT @Thebiscuitrule Just politely kicking in t...,0.0,56.0,#BBCforLockwoodandCo,NaN
2,1.663123e+18,2023-05-29 09:58:58+00:00,AndBenlovesrey,RT @lockwoodandco13 we're almost at 17K! DON'...,0.0,32.0,#BBCforLockwoodandCo,NaN
3,1.663062e+18,2023-05-29 05:57:39+00:00,Icebirdzzz,RT @Tatty_paints #SaveLockwoodandCo #BBCforLo...,0.0,32.0,#BBCforLockwoodandCo,NaN
4,1.663061e+18,2023-05-29 05:55:06+00:00,Icebirdzzz,RT @lavenderghostco OH MY GOD THOSE NUMBERS 🔥...,0.0,6.0,#BBCforLockwoodandCo,NaN
...,...,...,...,...,...,...,...,...
146935,1.664386e+18,2023-06-01 21:37:25+00:00,amazonlite2727,RT @Thebiscuitrule Just Cameron corpsing his ...,0.0,13.0,#WatchPartyatPortlandRow,NaN
146936,1.664379e+18,2023-06-01 21:11:35+00:00,DarkCyclePizza,RT @Thebiscuitrule Just Cameron corpsing his ...,0.0,13.0,#WatchPartyatPortlandRow,NaN
146937,1.663600e+18,2023-05-30 17:35:58+00:00,NinaLinkor598,RT @phinlands top searches‼️‼️‼️ #SaveLockwo...,0.0,47.0,#WatchPartyatPortlandRow,NaN
146938,1.662739e+18,2023-05-28 08:32:35+00:00,1sabellarry,RT @immzy_ OMG LOCKWOOD AND CO SEASON 2 SPOIL...,0.0,40.0,#WatchPartyatPortlandRow,NaN


In [68]:
working_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146940 entries, 0 to 146939
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tweet_id        146940 non-null  float64
 1   created_at      146940 non-null  object 
 2   user            146940 non-null  object 
 3   full_text       146940 non-null  object 
 4   favorite_count  146940 non-null  float64
 5   retweet_count   146940 non-null  float64
 6   hashtags        146940 non-null  object 
 7   tweet_df        0 non-null       float64
 8   retweet         146940 non-null  bool   
 9   tagged_users    146940 non-null  object 
dtypes: bool(1), float64(4), object(5)
memory usage: 10.2+ MB


In [48]:
working_df['full_text'] = working_df['full_text'].str.replace(r'"', '')
working_df['full_text']

0         RT @aDEPRACofficial  Good morning Locknation! ...
1         RT @Thebiscuitrule  Just politely kicking in t...
2         RT @lockwoodandco13  we're almost at 17K! DON'...
3         RT @Tatty_paints  #SaveLockwoodandCo #BBCforLo...
4         RT @lavenderghostco  OH MY GOD THOSE NUMBERS 🔥...
                                ...                        
146935    RT @Thebiscuitrule  Just Cameron corpsing his ...
146936    RT @Thebiscuitrule  Just Cameron corpsing his ...
146937    RT @phinlands  top searches‼️‼️‼️  #SaveLockwo...
146938    RT @immzy_  OMG LOCKWOOD AND CO SEASON 2 SPOIL...
146939    RT @wordywallflower  @bunnyellabell I visited ...
Name: full_text, Length: 146940, dtype: object

In [49]:
# Create a new column 'retweet' with default value False
working_df['retweet'] = False

# Check if 'full_text' starts with 'RT ' and set 'retweet' column accordingly
working_df.loc[working_df['full_text'].str.startswith('RT '), 'retweet'] = True

working_df['retweet'].value_counts()

True     74319
False    72621
Name: retweet, dtype: int64

In [78]:
#Create an anonymized tweet_id
#working_df['unique_id'] = str(working_df['tweet_id']) + working_df['created_at'] + working_df['user'] + str(working_df['retweet'])
working_df['unique_id'] = working_df.apply(lambda row: str(row['tweet_id']) + row['created_at'] + row['user'] + str(row['retweet']), axis=1)
working_df['tweet_id'] = working_df['unique_id'].apply(anonymize)
working_df['tweet_id'].unique

<bound method Series.unique of 0         cc636985
1         c405fe4f
2         e0a88e18
3         16409b86
4         2bae61eb
            ...   
146935    4656b9f6
146936    de3d38c8
146937    bd4573b3
146938    619abd4f
146939    3fc51db7
Name: tweet_id, Length: 146940, dtype: object>

In [52]:
working_df['tagged_users'] = working_df['full_text'].apply(lambda x: re.findall(r'@(\w+)', x))
working_df['tagged_users']

0                        [aDEPRACofficial]
1                    [Thebiscuitrule, bbc]
2                        [lockwoodandco13]
3                           [Tatty_paints]
4                        [lavenderghostco]
                        ...               
146935                    [Thebiscuitrule]
146936                    [Thebiscuitrule]
146937                         [phinlands]
146938                            [immzy_]
146939    [wordywallflower, bunnyellabell]
Name: tagged_users, Length: 146940, dtype: object

In [53]:
working_df.columns

Index(['tweet_id', 'created_at', 'user', 'full_text', 'favorite_count',
       'retweet_count', 'hashtags', 'tweet_df', 'retweet', 'tagged_users'],
      dtype='object')

In [79]:
#Create a table for network analysis
na_df = working_df[['tweet_id','user', 'retweet', 'tagged_users']]
#na_df = na_df.drop_duplicates()
na_df

,tweet_id,user,retweet,tagged_users
0,cc636985,CayleeGuy,True,[aDEPRACofficial]
1,c405fe4f,CayleeGuy,True,"[Thebiscuitrule, bbc]"
2,e0a88e18,AndBenlovesrey,True,[lockwoodandco13]
3,16409b86,Icebirdzzz,True,[Tatty_paints]
4,2bae61eb,Icebirdzzz,True,[lavenderghostco]
...,...,...,...,...
146935,4656b9f6,amazonlite2727,True,[Thebiscuitrule]
146936,de3d38c8,DarkCyclePizza,True,[Thebiscuitrule]
146937,bd4573b3,NinaLinkor598,True,[phinlands]
146938,619abd4f,1sabellarry,True,[immzy_]


In [80]:
#Create a function that would create a dataframe that shows the interaction
def get_interaction(df):
    func_df = pd.DataFrame(columns = ['tweet_id', 'from', 'to'])
    for x in range(len(df)):
        if df['retweet'][0]=='True':
            new_row = {'tweet_id':df['tweet_id'].iloc[x], 'from':df['tagged_users'].iloc[x][0], 'to': df['user'].iloc[x]}
            func_df = func_df.append(new_row, ignore_index = True)
        else:
            for user in df['tagged_users'].iloc[x]:
                new_row = {'tweet_id':df['tweet_id'].iloc[x], 'to':user, 'from': df['user'].iloc[x]}
                func_df = func_df.append(new_row, ignore_index = True)
    return func_df.drop_duplicates()


In [81]:
na_interac_df = get_interaction(na_df)
na_interac_df

,tweet_id,from,to
0,cc636985,CayleeGuy,aDEPRACofficial
1,c405fe4f,CayleeGuy,Thebiscuitrule
2,c405fe4f,CayleeGuy,bbc
3,e0a88e18,AndBenlovesrey,lockwoodandco13
4,16409b86,Icebirdzzz,Tatty_paints
...,...,...,...
164362,de3d38c8,DarkCyclePizza,Thebiscuitrule
164363,bd4573b3,NinaLinkor598,phinlands
164364,619abd4f,1sabellarry,immzy_
164365,3fc51db7,TheLempMansion,wordywallflower


In [67]:
interac_grouped_df = na_interac_df.groupby(['from','to','retweet'])['tweet_id'].count().reset_index().sort_values(by='tweet_id',ascending=False)
interac_grouped_df

,from,to,retweet,tweet_id
22215,kabitanaik29,lavenderghostco,False,517
9614,RebekahBlazek,netflix,False,467
230,2vargas2jpablos,lavenderghostco,False,457
17986,chocowhitel,lavenderghostco,False,436
4458,Godsgirljune,lavenderghostco,False,435
...,...,...,...,...
14235,amelia_argent,murdeebam,False,1
14234,amelia_argent,multistantara,False,1
14231,amelia_argent,miyaruepoetry,False,1
14228,amelia_argent,martinez_liya17,False,1


In [ ]:
if 'anonymized_user' in working_df.columns:
    working_df.drop('anonymized_user', axis=1, inplace=True)
user_df = pd.DataFrame(working_df['user'].unique(), columns=['user'])
user_df['anonymized_user'] = user_df['user'].apply(anonymize)
working_df = working_df.merge(user_df, on='user', how='left')


In [ ]:
working_df.columns

In [ ]:
working_df.to_csv('tweets.csv', index=False)

In [ ]:
len(working_df)

In [ ]:
working_df['created_date'] = pd.to_datetime(working_df['created_at'], utc=True)
working_df['week_number'] = working_df['created_date'].dt.week
working_df['week_number'].value_counts()

In [ ]:
def get_week_label(week_number):
    if week_number == 20:
        return 'week1'
    elif week_number == 21:
        return 'week2'
    elif week_number == 22:
        return 'week3'
    elif week_number == 23:
        return 'week4'
    elif week_number == 24:
        return 'week5'
    else:
        return 'week1'

In [ ]:
working_df['week_label'] = working_df['week_number'].apply(get_week_label)

In [ ]:
working_df.to_csv('tweeterhashtags/tweets.csv', index=False)

In [ ]:
working_df.value_counts('hashtags')